In [1]:
%matplotlib inline
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
from scipy import stats
from ast import literal_eval # 문자열 모형의 딕트를 스근하게 딕트로 바꾸어 준다. 
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD, evaluate

import warnings; warnings.simplefilter('ignore')

In [2]:
md = pd.read_csv('input/movies_metadata.csv')

In [3]:
md.isnull().sum()

adult                        0
belongs_to_collection    40972
budget                       0
genres                       0
homepage                 37684
id                           0
imdb_id                     17
original_language           11
original_title               0
overview                   954
popularity                   5
poster_path                386
production_companies         3
production_countries         3
release_date                87
revenue                      6
runtime                    263
spoken_languages             6
status                      87
tagline                  25054
title                        6
video                        6
vote_average                 6
vote_count                   6
dtype: int64

In [4]:
md.describe()

,revenue,runtime,vote_average,vote_count
count,4.546000e+04,45203.000000,45460.000000,45460.000000
mean,1.120935e+07,94.128199,5.618207,109.897338
std,6.433225e+07,38.407810,1.924216,491.310374
min,0.000000e+00,0.000000,0.000000,0.000000
25%,0.000000e+00,85.000000,5.000000,3.000000
50%,0.000000e+00,95.000000,6.000000,10.000000
75%,0.000000e+00,107.000000,6.800000,34.000000
max,2.787965e+09,1256.000000,10.000000,14075.000000


In [13]:
md.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
adult                    45466 non-null object
belongs_to_collection    4494 non-null object
budget                   45466 non-null object
genres                   45466 non-null object
homepage                 7782 non-null object
id                       45466 non-null object
imdb_id                  45449 non-null object
original_language        45455 non-null object
original_title           45466 non-null object
overview                 44512 non-null object
popularity               45461 non-null object
poster_path              45080 non-null object
production_companies     45463 non-null object
production_countries     45463 non-null object
release_date             45379 non-null object
revenue                  45460 non-null float64
runtime                  45203 non-null float64
spoken_languages         45460 non-null object
status                   45379 non-null objec

In [35]:
vote_count = md[md['vote_count'].notnull()]['vote_count'].astype('int') 
vote_average = md[md['vote_average'].notnull()]['vote_average'].astype('int')
m = vote_count.quantile(0.95)
C = vote_average.mean()
C

5.244896612406511

In [37]:
md['release_date'].apply(lambda x : str(x).split('-')[0] if x != np.nan else np.nan)

0        1995
1        1995
2        1995
3        1995
4        1995
5        1995
6        1995
7        1995
8        1995
9        1995
10       1995
11       1995
12       1995
13       1995
14       1995
15       1995
16       1995
17       1995
18       1995
19       1995
20       1995
21       1995
22       1995
23       1995
24       1995
25       1995
26       1995
27       1995
28       1995
29       1995
         ... 
45436    2010
45437    2017
45438    2015
45439    1944
45440    2007
45441    2002
45442    1905
45443    1901
45444    1905
45445    1906
45446    1909
45447    1904
45448    1904
45449    2005
45450    1900
45451    1900
45452    1981
45453    2017
45454    2015
45455    1972
45456    1946
45457    2000
45458    2000
45459    1995
45460    1991
45461     nan
45462    2011
45463    2003
45464    1917
45465    2017
Name: release_date, Length: 45466, dtype: object

In [40]:
qualified = md[(md['vote_count'].notnull()) & (md['vote_average'].notnull()) & (md['vote_count'] >= m)][['title', 'year', 'vote_count', 'vote_average', 'popularity', 
                                                                                                         'genres']]
qualified['vote_count'] = qualified['vote_count'].astype(int)
qualified['vote_average'] = qualified['vote_average'].astype(int)

In [30]:
def weight_rate(x):
    v = x['vote_count']
    r = x['vote_average']
    return (v / (v + m)*r) + (m/(m+v)*C)

In [43]:
qualified['wr'] = qualified.apply(weight_rate, axis=1)

In [44]:
qualified = qualified.sort_values(by='wr', ascending=False)

In [45]:
qualified.head(10)

,title,year,vote_count,vote_average,popularity,genres,wr
15480,Inception,2010,14075,8,29.1081,"[Action, Thriller, Science Fiction, Mystery, A...",7.917588
12481,The Dark Knight,2008,12269,8,123.167,"[Drama, Action, Crime, Thriller]",7.905871
22879,Interstellar,2014,11187,8,32.2135,"[Adventure, Drama, Science Fiction]",7.897107
2843,Fight Club,1999,9678,8,63.8696,[Drama],7.881753
4863,The Lord of the Rings: The Fellowship of the Ring,2001,8892,8,32.0707,"[Adventure, Fantasy, Action]",7.871787
292,Pulp Fiction,1994,8670,8,140.95,"[Thriller, Crime]",7.868660
314,The Shawshank Redemption,1994,8358,8,51.6454,"[Drama, Crime]",7.864000
7000,The Lord of the Rings: The Return of the King,2003,8226,8,29.3244,"[Adventure, Fantasy, Action]",7.861927
351,Forrest Gump,1994,8147,8,48.3072,"[Comedy, Drama, Romance]",7.860656
5814,The Lord of the Rings: The Two Towers,2002,7641,8,29.4235,"[Adventure, Fantasy, Action]",7.851924


In [50]:
md.apply(lambda x: pd.Series(x['genres']), axis=1).stack().reset_index(level=0)

,level_0,level_1,0
0,0,0,Animation
1,0,1,Comedy
2,0,2,Family
3,1,0,Adventure
4,1,1,Fantasy
5,1,2,Family
6,2,0,Romance
7,2,1,Comedy
8,3,0,Comedy
9,3,1,Drama


In [51]:
md.apply(lambda x: pd.Series(x['genres']), axis=1).stack().reset_index(level=0)

,level_0,0
0,0,Animation
1,0,Comedy
2,0,Family
0,1,Adventure
1,1,Fantasy
2,1,Family
0,2,Romance
1,2,Comedy
0,3,Comedy
1,3,Drama


In [58]:
s = md.apply(lambda x: pd.Series(x['genres']), axis=1).stack().reset_index(level=1, drop=True)
s

0              Animation
0                 Comedy
0                 Family
1              Adventure
1                Fantasy
1                 Family
2                Romance
2                 Comedy
3                 Comedy
3                  Drama
3                Romance
4                 Comedy
5                 Action
5                  Crime
5                  Drama
5               Thriller
6                 Comedy
6                Romance
7                 Action
7              Adventure
7                  Drama
7                 Family
8                 Action
8              Adventure
8               Thriller
9              Adventure
9                 Action
9               Thriller
10                Comedy
10                 Drama
              ...       
45446             Comedy
45449          Animation
45449             Family
45450            Fantasy
45450             Action
45450           Thriller
45451             Comedy
45451            Fantasy
45452        Documentary


In [60]:
s.name = 'genres'
s_gen = md.drop('genres', axis=1).join(s)

In [69]:
def chart_bar(genre, percenTage=0.85):
    df = s_gen[s_gen['genres'] == genre]
    vote_count = df[df['vote_count'].notnull()]['vote_count'].astype(int)
    vote_average = df[df['vote_average'].notnull()]['vote_average'].astype(int)
    m = vote_count.quantile(percenTage)
    c = vote_average.mean()

    qualified = df[(df['vote_count'].notnull()) & (df['vote_count'] >= m) & (df['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity']]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    qualified['wr'] = qualified.apply(lambda x: (x['vote_count'] / (x['vote_count'] + m)*x['vote_average']) + (m/(m+x['vote_count'])*c), axis=1) 
    qualified = qualified.sort_values('wr', ascending=False)
    return qualified

,title,year,vote_count,vote_average,popularity,wr
10309,Dilwale Dulhania Le Jayenge,1995,661,9,34.457,8.565285
351,Forrest Gump,1994,8147,8,48.3072,7.971357
876,Vertigo,1958,1162,8,18.2082,7.811667


In [71]:
chart_bar('Romance').head(3)


,title,year,vote_count,vote_average,popularity,wr
10309,Dilwale Dulhania Le Jayenge,1995,661,9,34.457,8.565285
351,Forrest Gump,1994,8147,8,48.3072,7.971357
876,Vertigo,1958,1162,8,18.2082,7.811667


In [72]:
chart_bar('Adventure').head(3)

,title,year,vote_count,vote_average,popularity,wr
15480,Inception,2010,14075,8,29.1081,7.906526
22879,Interstellar,2014,11187,8,32.2135,7.883426
4863,The Lord of the Rings: The Fellowship of the Ring,2001,8892,8,32.0707,7.854939
